# FINM HW3

## Problem 1

## Problem 2

### 1. The Factors 

Calculate their univariate performance statistics:
* Mean
* Volatility
* Sharpe
* VaR

## Problem 3

## Problem 4

## Problem 5 (Extensions)